In [1]:
import xmltodict as xd
import pandas as pd
import os, glob
import mio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pathlib

In [2]:
def parse_model(dae_file:str) -> dict:
    " get point and vertex list for one model"
    with open(dae_file) as fin:
        s = fin.read()
    d = xd.parse(s)
    
    geometries = d['COLLADA']['library_geometries']['geometry']
    
    res = {}
    for geo in geometries:
        geo_id = geo['@id']

        # float list
        f_list = geo['mesh']['source']['float_array']['#text'].split()
        f_list = [float(f) for f in f_list]

        # vertex list
        v_list = geo['mesh']['triangles']['p'].split()
        v_list = [int(n) for n in v_list]
        # write result
        
        res[geo_id] = (f_list, v_list)
    
    return res

In [3]:
def comb_f_v(f_list, v_list):
    va = np.array(v_list)
    va = va.reshape(len(v_list)//3, 3)
    v = pd.DataFrame(va, columns=['v0', 'v1', 'v2'])
    
    fa = np.array(f_list)
    fa = fa.reshape(len(f_list)//3, 3)
    f = pd.DataFrame(fa, columns=['x', 'y', 'z'])
    
    assert max(v.v0.max(), v.v1.max(), v.v2.max()) +1 == len(f)
    
    # combine the lists
    res = (
        v.join(f, on='v0').rename(columns={'x':'x0', 'y':'y0', 'z':'z0'})
         .join(f, on='v1').rename(columns={'x':'x1', 'y':'y1', 'z':'z1'})
         .join(f, on='v2').rename(columns={'x':'x2', 'y':'y2', 'z':'z2'})
         .drop(columns=['v0', 'v1', 'v2'])
    )
    
    return res

In [4]:
def dae2df(dae_path):
    dae_path = pathlib.Path(dae_path)
    dic = parse_model(dae_path)
    ldf = []
    for key in dic:
        f_list, v_list = dic[key]
        df = comb_f_v(f_list, v_list)
        ldf.append(df)
    
    res = pd.concat(ldf)
    res.insert(0, 'model', dae_path.name)
    return res

In [5]:
def get_model_list(kml_path:str)->pd.DataFrame:
    " read kml file, get path and anchor coordinates for every model"
    with open(kml_path) as fin:
        s = fin.read()
    kmld =  xd.parse(s)
    rows = []
    for pm in kmld['kml']['Document']['Placemark'][0:]:
        model = pm['Model']
        row = {}
        row['model'] = os.path.basename(model['Link']['href'])
        sLon = model['Location']['longitude']
        sLat = model['Location']['latitude']
        row['swiss_lon'] = float(sLon)
        row['swiss_lat'] = float(sLat)
        row['swiss_x'], row['swiss_y'] = mio.wgs_swiss(sLon, sLat)
        rows.append(row)
    df = pd.DataFrame(rows)
    
    mdir = str(pathlib.Path(kml_path).parent / 'models')
    res = (
        df.assign(model_path = lambda d : mdir + '\\' + d.model)
        .set_index('model')
    )
    return res

In [6]:
# create datafrme with all triangles
#
def make_tri_tab(kml):
    p_kml = pathlib.Path(kml)
    mod_list = get_model_list(p_kml)
    ldf = []
    i = 0
    for ind, row in mod_list.iterrows():
        i += 1
        mio.show_perc(i, len(mod_list), 100)
        try:
            tris = dae2df(row.model_path)
            tris.insert(0, 'kml', p_kml.name)
            tris.insert(2, 'swiss_x', row.swiss_x)
            tris.insert(3, 'swiss_y', row.swiss_y)
            ldf.append(tris)
        except:
            print(f'Problem with {row.model_path}')
    df = pd.concat(ldf)

    # create a geo dataframe
    vec = (
        pd.DataFrame()
        .assign(kml = df.kml)
        .assign(model = df.model)
        .assign(x0 = df.x0 + df.swiss_x, y0 = df.y0 + df.swiss_y, z0 = df.z0)
        .assign(x1 = df.x1 + df.swiss_x, y1 = df.y1 + df.swiss_y, z1 = df.z1)
        .assign(x2 = df.x2 + df.swiss_x, y2 = df.y2 + df.swiss_y, z2 = df.z2)
    )
    geom = [Polygon( [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]) for t in zip(vec.x0, vec.y0, vec.x1, vec.y1, vec.x2, vec.y2)]
    gdf = gpd.GeoDataFrame(vec, geometry=geom)
    # filter out vertical walls
    res = gdf[gdf.geometry.area > 0]
    return gdf

In [7]:
# MAIN
tag = '1091-13'
kml = f"C:/Buildings2.0/KML/{tag}/{tag}.kml"
gdf = make_tri_tab(kml)
mio.write_tab(gdf, f'result/{tag}.tab')

1.71% 3.42% 5.12% 6.83% 8.54% 10.25% 11.96% 13.67% 15.37% 17.08% 18.79% 20.5% 22.21% 23.92% 25.62% 27.33% 29.04% 30.75% 32.46% 34.16% 35.87% 37.58% 39.29% 41.0% 42.71% 44.41% 46.12% 47.83% 49.54% 51.25% 52.96% 54.66% 56.37% 58.08% 59.79% 61.5% 63.2% 64.91% 66.62% 68.33% 70.04% 71.75% 73.45% 75.16% 76.87% 78.58% 80.29% 82.0% 83.7% 85.41% 87.12% 88.83% 90.54% 92.24% 93.95% 95.66% 97.37% 99.08% 345700 rows of type MultiPolygon written to mapinfo file.
